In [ ]:
%env CUDA_LAUNCH_BLOCKING=1

In [ ]:
from experiments.attn.victory import *

In [ ]:
B = 1024
worlds = Hex.initial(B, boardsize=9)

In [ ]:
%%snakeviz
with tqdm(total=T) as pbar:
    for t, b in zip(range(100), batchgen(B, B)):
        outputs = model(b.worlds.obs)

        loss = F.nll_loss(outputs.reshape(B, -1), b.actions)

        opt.zero_grad()
        loss.backward()
        opt.step()

        pbar.update(1)
        pbar.set_description(f'{loss:.2f}')

In [ ]:
!nvidia-smi

In [ ]:
worlds.obs.sum(-1).reshape(worlds.n_envs, -1).gather(1, boards.actions[:, None]).min()